# Churn operadora Machine Learning (Python)

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import requests
import seaborn as sns
sns.set()
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

import re

# IQR/Z score.
from scipy.stats import stats

# Seleción de variables.
from sklearn.feature_selection import SelectKBest

# Modelos Selección.
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# Modelos.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeClassifierCV
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold

# Preprocesado.
from sklearn.preprocessing import MinMaxScaler

from imblearn.over_sampling import SMOTE
import pickle
# Métricas.
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_score, recall_score, roc_curve, classification_report

## Cargar los datasets y construir un único tablón analítico con todas las variables

In [2]:
dic_clientes = pd.read_csv("clientes_diciembre.csv", sep = "|", parse_dates = ["antiguedad"])
dic_consumos = pd.read_csv("consumos_diciembre.csv", sep = "|")
dic_fin = pd.read_csv("financiacion_diciembre.csv", sep = "|")
dic_prod = pd.read_csv("productos_diciembre.csv", sep = "|")

In [3]:
print('Tamaño de tabla de consumos diciembre:',dic_consumos.shape[0])
print('Tamaño de tabla de clientes diciembre:',dic_clientes.shape[0])
print('Tamaño de tabla de productos diciembre:',dic_prod.shape[0])
print('Tamaño de tabla de financiación diciembre:',dic_fin.shape[0])

Tamaño de tabla de consumos diciembre: 95467
Tamaño de tabla de clientes diciembre: 95467
Tamaño de tabla de productos diciembre: 95467
Tamaño de tabla de financiación diciembre: 24198


In [4]:
print('Clientes en tabla consumos\n',dic_clientes['id'].isin(dic_consumos['id']).value_counts())
print('Clientes en tabla productos\n',dic_clientes['id'].isin(dic_prod['id']).value_counts())
print('Clientes en tabla financiacion\n', dic_clientes['id'].isin(dic_fin['id']).value_counts())

Clientes en tabla consumos
 True    95467
Name: id, dtype: int64
Clientes en tabla productos
 True    95467
Name: id, dtype: int64
Clientes en tabla financiacion
 False    71269
True     24198
Name: id, dtype: int64


In [5]:
dic_cl_con = pd.merge(dic_clientes, dic_consumos, on = "id", how = "left")
dic_cl_con_f = pd.merge(dic_cl_con, dic_fin, on= "id", how = "left")
dic_full = pd.merge(dic_cl_con_f, dic_prod, on = "id", how = "left")

In [6]:
dic_full.head()

,id,edad,facturacion,antiguedad,provincia,num_lineas,num_dt,incidencia,num_llamad_ent,num_llamad_sal,mb_datos,seg_llamad_ent,seg_llamad_sal,financiacion,imp_financ,descuentos,conexion,vel_conexion,TV
0,1,63,216.028109,2018-11-23 08:48:00,La Rioja,5,NaN,NaN,110,79,10897,12806,13751,NaN,NaN,NaN,FIBRA,50MB,tv-futbol
1,2,84,255.830842,2017-08-22 03:19:00,Vizcaya,3,NaN,NaN,189,89,18657,6499,10862,NaN,NaN,SI,FIBRA,600MB,tv-futbol
2,3,66,135.768153,2001-12-27 13:50:00,Albacete,4,NaN,NaN,129,30,15511,17013,16743,NaN,NaN,SI,ADSL,35MB,tv-futbol
3,4,69,255.658527,2015-08-08 10:53:00,Lugo,4,NaN,NaN,51,52,12670,3393,6771,NaN,NaN,NaN,FIBRA,200MB,tv-familiar
4,5,30,22.302845,1997-08-29 02:19:00,Tarragona,2,2.0,NaN,183,3,23756,18436,4485,NaN,NaN,NaN,ADSL,10MB,tv-futbol


In [7]:
dic_full.shape

(95467, 19)

In [8]:
en_clientes = pd.read_csv("clientes_enero.csv", sep = "|", parse_dates = ["antiguedad"])
en_consumos = pd.read_csv("consumos_enero.csv", sep = "|")
en_fin = pd.read_csv("financiacion_enero.csv", sep = "|")
en_prod = pd.read_csv("productos_enero.csv", sep = "|")

In [9]:
print('Tamaño de tabla de consumos enero:',en_consumos.shape[0])
print('Tamaño de tabla de clientes enero:',en_clientes.shape[0])
print('Tamaño de tabla de productos enero:',en_prod.shape[0])
print('Tamaño de tabla de financiación enero:',en_fin.shape[0])

Tamaño de tabla de consumos enero: 92711
Tamaño de tabla de clientes enero: 92711
Tamaño de tabla de productos enero: 92711
Tamaño de tabla de financiación enero: 25332


In [10]:
print('Clientes en tabla consumos\n',en_clientes['id'].isin(en_consumos['id']).value_counts())
print('Clientes en tabla productos\n',en_clientes['id'].isin(en_prod['id']).value_counts())
print('Clientes en tabla financiacion\n', en_clientes['id'].isin(en_fin['id']).value_counts())

Clientes en tabla consumos
 True    92711
Name: id, dtype: int64
Clientes en tabla productos
 True    92711
Name: id, dtype: int64
Clientes en tabla financiacion
 False    67379
True     25332
Name: id, dtype: int64


In [11]:
en_cl_con = pd.merge(en_clientes, en_consumos, on = "id", how = "left")
en_cl_con_f = pd.merge(en_cl_con, en_fin, on = "id", how = "left")
en_full = pd.merge(en_cl_con_f, en_prod, on = "id", how = "left")

In [12]:
## Valores únicos
print('Enero: ',len(en_full['id'].unique()))
print('Diciembre: ',len(dic_full['id'].unique()))

Enero:  92711
Diciembre:  95467


In [13]:
dic_full["mes"] = "dic"
en_full["mes"] = "en"

   - Se observa que en enero hubo una disminución de 3000 clientes aproximadamente. Por tanto, la empresa perdió más clientes de los que ganó. 
        - __7085 clientes abandonaron la empresa.__

In [14]:
len(dic_full) - dic_full.id.isin(en_full.id).sum()

7085

   - Creación de la variable target. Se buscan los "id" de enero que coinciden con los de diciembre, asignando 0 si son idénticos o 1 si no lo son.

In [ ]:
churn = pd.DataFrame()
churn['churn'] = np.where(dic_full.id.isin(en_full.id) == True, 0, 1)
churn['id'] =dic_full['id'] 

In [ ]:
####  Antes de concatenar las cosechas, voy a transformar la variable antigüedad
dic_full['duracion'] = (datetime(2019,12,31) - dic_full["antiguedad"])
en_full['duracion'] = (datetime(2020,1,31) - en_full["antiguedad"])

In [ ]:
## Concatenamos las dos cosechas y normalizamos todas las variables

data = pd.concat([dic_full, en_full], sort = False, ignore_index=True)
data.columns= data.columns.str.lower()
data.head()

In [ ]:
# Transformación de la variable antigüedad a días, horas... Esta parte podría hacerse en la parte de FE
data["duracion"] = data.duracion.apply(lambda x: int(x.days))

# Month:
data["month"] = data.antiguedad.apply(lambda x: x.strftime("%B"))

# Lo mismo con día de la semana. 
data["dia"] = data.antiguedad.apply(lambda x: x.strftime("%A"))

# Recoger la info de la hora.
data["hora"] = data.antiguedad.apply(lambda x: int(x.hour))

# Categórica para poder tener interpretabilidad.
data["hora_bin"] = pd.cut(data["hora"], bins = [-1,6,12,18,23], labels = ["Madrugada", "Mañana", "Tarde", "Noche"])

# Se elimina la variable original pues ya está convertida a numérica.
data.drop(columns = "antiguedad", inplace = True)

# Pre procesado y  limpieza de los datos



In [ ]:
data.info()

In [ ]:
cat_cols= data.select_dtypes(include=['object','category']).columns
num_cols = data.select_dtypes(exclude=['object','category']).columns

In [ ]:
data[cat_cols].describe()

In [ ]:
data[num_cols].describe()

##  Tratamiento de nulos

In [ ]:
data.isnull().sum()

In [ ]:
## Las variables incidencia, financiación, descuentos cuando están a nulo significa que es NO (0)

var_no = ['incidencia','financiacion','descuentos']

for i in var_no:
    data[i].fillna('NO',inplace=True)

In [ ]:
## Las variables num_dt, imp_financ cuando están a nulo significa que es el valor es 0.
var_cero = ['num_dt','imp_financ']

for i in var_cero:
    data[i].fillna(0,inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
data['vel_conexion'].value_counts().plot(kind='bar')

In [ ]:
data.dtypes

In [ ]:
data[data['vel_conexion']=='?']

In [ ]:
data[data['vel_conexion']=='1']

In [ ]:
data[data['vel_conexion']=='16598']

In [ ]:
adsl = data.query('conexion == "ADSL"')
adsl.groupby('vel_conexion')['conexion'].count().sort_values(ascending=False)

In [ ]:
# Vemos que dentro de las conexiones adsl, la velocidad de conexión más repetida es la de 20 MB 
data['vel_conexion'] = data['vel_conexion'].replace('1', '20MB')

In [ ]:
fibra = data.query('conexion == "FIBRA"')
fibra.groupby('vel_conexion')['conexion'].count().sort_values(ascending=False)

In [ ]:
# Vemos que dentro de las conexiones fibra, la velocidad de conexión más repetida es la de 200 MB 
data['vel_conexion'] = data['vel_conexion'].replace('?', '200MB')
data['vel_conexion'] = data['vel_conexion'].replace('16598', '200MB')

In [ ]:
## Faltan por rellenar unos pocos nulos en la variable conexión y en la variable vel_conexion

print(data.conexion.unique())

print(data.conexion.value_counts())

## La probabilidad de rellenar el valor con ADSL o FIBRA es prácticamente el mismo. En este caso, podríamos rellenarlo con la moda ( ADSL ) o incluso eliminar las dos líneas

data['conexion'].fillna('ADSL',inplace=True)

print(data.vel_conexion.unique())

print(data.vel_conexion.value_counts())

## En el caso de la velocidad, rellenaremos con el valor 200MB (moda). En este caso, se podría haber rellenado también de varias maneras. Normalmente en estos casos, tenemos alguna información más de negocio (velocidad más probable...)

data['vel_conexion'].fillna('200MB',inplace=True)

## Vemos también 3 valores que trataremos después

In [ ]:
#Variables numéricas
fig, axs = plt.subplots(ncols=5, nrows=3, figsize=(20, 10))
axs = axs.flatten() # 

index = 0
for k,v in data[num_cols].items(): 
    if  (k == 'permanencia'):
        sns.countplot(v, ax=axs[index])
    else:
        sns.distplot(v, bins=20, ax=axs[index])
    index += 1

plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

In [ ]:
#Variables categóricas
fig, axs = plt.subplots(ncols=5, nrows=3, figsize=(20, 10))
axs = axs.flatten() # 

index = 0
for k,v in data[cat_cols].items(): 
    sns.countplot(v, ax=axs[index])
    index += 1

plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=5.0)

## Transformación variables a numéricas

In [ ]:
data.dtypes

In [ ]:
variables_dummies = ['conexion','tv']
variables_binarias = ['financiacion','descuentos','incidencia']

In [ ]:
for k in variables_dummies:
    dummies = pd.get_dummies(data[k],prefix=k)
    data = pd.concat([data, dummies], axis=1)
    data.drop(k, axis=1, inplace=True)

In [ ]:
for j in variables_binarias:
    data[j] = [1 if x == 'SI' else 0 for x in data[j]]

In [ ]:
data["vel_conexion"] = data.vel_conexion.apply(lambda x: int(re.sub("MB", "", x)))
data.vel_conexion.head()

In [ ]:
## Variables nuevas

data.month.replace(('January','February','March','April','May','June','July','August','September','October','November','December'),
                      (1,2,3,4,5,6,7,8,9,10,11,12),inplace=True)

data.dia.replace(('Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'),
                      (1,2,3,4,5,6,7),inplace=True)

data.hora_bin.replace(('Madrugada','Mañana','Tarde','Noche'),
                      (1,2,3,4),inplace=True)

In [ ]:
## Obtenemos la comunidad autónoma de cada provincia
url = "https://es.wikipedia.org/wiki/Provincia_(Espa%C3%B1a)"
html = requests.get(url).content
df_list = pd.read_html(html)
ccaa = df_list[1]
ccaa = ccaa[["Comunidad autónoma", "Provincia"]]
ccaa.columns =ccaa.columns.str.lower()
ccaa = ccaa.rename(columns = {'comunidad autónoma': 'com_aut'})
ccaa.provincia.replace(('Baleares'),('Islas Baleares'), inplace=True)

In [ ]:
data = pd.merge(data,ccaa,how='left',on='provincia')

In [ ]:
data

In [ ]:
data.drop('provincia', axis=1, inplace=True )

In [ ]:
len(data.com_aut.unique())

In [ ]:
data.com_aut.unique()

In [ ]:
data.com_aut.replace(('La Rioja', 'País Vasco', 'Castilla-La Mancha', 'Galicia',
                       'Cataluña', 'Andalucía', 'Comunidad Valenciana',
                       'Comunidad de Madrid', 'Islas Canarias', 'Castilla y León', 'Aragón',
                       'Navarra', 'Extremadura', 'Asturias',
                       'Islas Baleares', 'Cantabria', 'Región de Murcia'),
                        (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17),inplace=True)

In [ ]:
data.com_aut = data.com_aut.astype(int)

In [ ]:
data

In [ ]:
list_correlacion = ['edad', 'facturacion', 'num_lineas', 'num_dt', 'incidencia',
       'num_llamad_ent', 'num_llamad_sal', 'mb_datos', 'seg_llamad_ent',
       'seg_llamad_sal', 'financiacion', 'imp_financ', 'descuentos',
       'vel_conexion','duracion', 'month', 'dia', 'hora',
       'hora_bin','com_aut']

df_correlacion = data[list_correlacion]
correlation_mat = df_correlacion.corr()
sns.heatmap(correlation_mat)
plt.show()

## Vemos correlación entre las variables num_lineas y facturacion y seg_llamd_sali y financiación, aunque muy baja

In [ ]:
data.dtypes

In [ ]:
for i in list_correlacion:
    sns.boxplot(x=data[i])
    print(i)
    plt.show()
    
    
## Vemos posibles outliers en vel conexión y num_lineas. Vamos a comprobarlo con z_score    

In [ ]:
threshold = 3
z = np.abs(stats.zscore(data['vel_conexion']))
print(z)
print(np.where(z > 3))

In [ ]:
threshold = 3
z = np.abs(stats.zscore(data['num_lineas']))
print(z)
print(np.where(z > 3))

## Hay 3 valores que están fuera del rango. Podemos eliminar esas filas

data.drop([2580,8657,36987], axis=0,inplace=True)

In [ ]:
### Todos los valores cumplen con el estandar

In [ ]:
data.dtypes

In [ ]:
df_dic = pd.merge(data[data['mes']=='dic'], churn, on='id', how='left')

In [ ]:
df_dic.drop('mes', axis=1, inplace=True)
df_dic.drop('hora_bin', axis=1, inplace=True)
df_dic.set_index('id', inplace=True)

In [ ]:
df_dic

In [ ]:
## Las variables más importantes del modelo son num_dt, incidencia, descuentos, financiacion, importe financiacion

In [ ]:
corr = abs(df_dic.corr())
corr[['churn']].sort_values(by = 'churn',ascending = False).style.background_gradient()

In [ ]:
def proporciones_final (var,target,df):

    proporcion = pd.DataFrame()
    
    proporcion['%fugas'] = df[target].groupby(df[var]).mean()*100
    proporcion['Conteo'] = df[target].groupby(df[var]).count()
    proporcion= proporcion.round(3)   
    proporcion_filtered = proporcion[(proporcion['%fugas']>0) & (proporcion['Conteo']>10)]  
        
    if len(proporcion_filtered)<100 and len(proporcion_filtered)>1:
        fig = plt.figure()
        ax = proporcion_filtered['Conteo'].plot(kind='bar',grid=True)
                
        ax2 = ax.twinx()
        ax2.plot(proporcion_filtered['%fugas'].values, linestyle='-', linewidth=2.0,color='g')
        plt.ylim(0, 100) # modificación.
        plt.tight_layout()   
        
    else:        
        proporcion_filtered.reset_index(inplace=True)
        sns.lmplot(x = var,y ='%fugas', data=proporcion_filtered,fit_reg=True,ci=None)

In [ ]:
for i in df_dic.columns:
    proporciones_final (i, 'churn', df_dic)

# Muestrear los datos, construyendo un dataset de train y test

   - Muestreo estratificado en función de la variable respuesta para que en los splits sus categorías estén bien representadas.
       - Split 80/20.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split (df_dic.drop('churn', axis=1), 
                                                     df_dic.churn,
                                                     test_size = 0.2,
                                                     random_state = 0,
                                                     stratify = df_dic.churn)

# Construir un modelo analítico de clasificación que sea capaz de predecir cuando un cliente se fuga de la empresa

In [ ]:
def saca_metricas(y_real, y_pred): 
    false_positive_rate, recall, thresholds = roc_curve(y_real, y_pred)
    roc_auc = auc(false_positive_rate, recall)
    print('- AUC: {roc_auc}')
    plt.plot(false_positive_rate, recall, 'b')
    plt.plot([0, 1], [0, 1], 'r--')
    plt.title('AUC = %0.2f' % roc_auc)

In [ ]:
import mlflow

In [ ]:
mlflow.set_experiment("ejercicio_churn")

In [ ]:
# from sklearn.tree import DecisionTreeClassifier

# Modelo y predicciones:
tree_mod = DecisionTreeClassifier(criterion="gini").fit(X_train, y_train)
tree_pred = tree_mod.predict(X_test)

# Métricas:
saca_metricas(y_test, tree_pred)

In [ ]:
with mlflow.start_run(run_name='decission_tree'):

    # predict_proba returns [prob_negative, prob_positive], so slice the output with [:, 1]
    accuracy = accuracy_score(y_test, tree_pred)
    false_positive_rate, recall, thresholds = roc_curve(y_test, tree_pred)
    roc_auc = auc(false_positive_rate, recall)
   
    # Use the area under the ROC curve as a metric.
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('roc_auc', roc_auc)
    # log model
    mlflow.sklearn.log_model(tree_mod,'tree_ntic')
    # save model
    mlflow.sklearn.save_model(tree_mod, 'tree.pkl',
                              serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_PICKLE)

In [ ]:
classifier2 = RandomForestClassifier().fit(X_train, y_train)
pred2 = classifier2.predict(X_test)

In [ ]:
saca_metricas(y_test, pred2)

In [ ]:
with mlflow.start_run(run_name='random_forest'):

    # predict_proba returns [prob_negative, prob_positive], so slice the output with [:, 1]
    accuracy = accuracy_score(y_test, pred2)
    false_positive_rate, recall, thresholds = roc_curve(y_test, pred2)
    roc_auc = auc(false_positive_rate, recall)
   
    # Use the area under the ROC curve as a metric.
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('roc_auc', roc_auc)
    # log model
    mlflow.sklearn.log_model(classifier2,'rf_ntic')
    # save model
    mlflow.sklearn.save_model(tree_mod, 'rf.pkl',
                              serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_PICKLE)

In [ ]:
classifier3 = LinearSVC(penalty='l1', dual= False).fit(X_train, y_train)
pred3 = classifier3.predict(X_test)

In [ ]:
saca_metricas(y_test, pred3)

In [ ]:
classifier4 = LogisticRegression(max_iter=10000).fit(X_train, y_train)
pred4 = classifier4.predict(X_test)

In [ ]:
saca_metricas(y_test, pred4)

In [ ]:
print(classification_report(y_test, pred4))

In [ ]:
classifier5 = GaussianNB().fit(X_train, y_train)
pred5 = classifier5.predict(X_test)

In [ ]:
saca_metricas(y_test, pred5)

In [ ]:
with mlflow.start_run(run_name='gaussiano'):

    # predict_proba returns [prob_negative, prob_positive], so slice the output with [:, 1]
    accuracy = accuracy_score(y_test, pred5)
    false_positive_rate, recall, thresholds = roc_curve(y_test, pred5)
    roc_auc = auc(false_positive_rate, recall)
   
    # Use the area under the ROC curve as a metric.
    mlflow.log_metric('accuracy', accuracy)
    mlflow.log_metric('roc_auc', roc_auc)
    # log model
    mlflow.sklearn.log_model(classifier5,'gaus_ntic')
    # save model
    mlflow.sklearn.save_model(tree_mod, 'gaus.pkl',
                              serialization_format=mlflow.sklearn.SERIALIZATION_FORMAT_PICKLE)

In [ ]:
print(classification_report(y_test, pred5))

In [ ]:
# Validación cruzada

cv = cross_val_score(
    classifier2,
    X_train, 
    y_train,
    scoring = "roc_auc",
    cv = 5
)
print(cv)
print("CV ROC:", cv.mean(), np.std(cv))

#### Selección de variables

In [ ]:
imp = {}
for i in range(len(X_train.columns)):
    imp[X_train.columns[i]] = [classifier2.feature_importances_[i]]
pd.DataFrame.from_dict(imp, orient="index", columns=["Importance"]).sort_values("Importance", ascending=False).head(20).style.background_gradient()

#### Podemos comparar varios modelos/hiperparámetros

![](recursive.png)

In [ ]:
models_rcv = {
    "XGB": XGBClassifier(n_jobs=-1, n_estimators=30, random_state=1234),
    "RF": RandomForestClassifier(n_estimators =30, n_jobs=-1, random_state=1234),
    "Tree": DecisionTreeClassifier(random_state=1234),
    "Log": LogisticRegression(solver="newton-cg", penalty="l2"),
}

def rskf_comparison(models, X_train, y_train):
    
    results = []
    names = []

    for k, v in models.items():
    
        rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1234) 
        cv_scores = cross_val_score(v, X_train, y_train, scoring='roc_auc', cv=rskf, n_jobs=-1)

        results.append(cv_scores)
        names.append(k)

        print(k)
        print('CV AUC: %.5f +/- %.5f' % (np.mean(cv_scores), np.std(cv_scores)))
        print('-------------------------')
        
    
    return(results, names)


results, names = rskf_comparison(models_rcv, X_train, y_train)
plt.figure(figsize=(10, 6))
comparison = plt.boxplot(results)
plt.xticks(np.arange(1,len(names)+1),names)
plt.show(comparison)

In [ ]:
## Tuning hiperparámetros

In [ ]:
grid_param = {
    'n_estimators': [100,300,500],
    'criterion': ['gini', 'entropy'],
    'max_depth': [16,20,22,24],
    'max_features': ['auto', 'sqrt'],

}

In [ ]:
stratified_kfold = StratifiedKFold(n_splits=5,
                                       shuffle=True,
                                       random_state=11)

In [ ]:
model_grid = GridSearchCV(estimator=classifier2,
                     param_grid=grid_param,
                     scoring='roc_auc',
                     cv=stratified_kfold,
                     n_jobs=-1)

In [ ]:
print(classification_report(y_test,pred2))

In [ ]:
classifier2 = RandomForestClassifier(criterion = 'entropy', n_estimators= 500, max_depth = 16, max_features= 'auto').fit(X_train, y_train)
pred2 = classifier2.predict(X_test)

In [ ]:
print(classification_report(y_test,pred2))

In [ ]:
## Se podría mejorar el modelo balanceando datos, trameando variables, creando variables nuevas...

In [ ]:
print("Before OverSampling, counts of label '1': {}".format(sum(df_dic.churn==1)))
print("Before OverSampling, counts of label '0': {}".format(sum(df_dic.churn==0)))

sm = SMOTE(sampling_strategy=0.4, random_state=2)
df_dic_res, y_res = sm.fit_resample(df_dic.drop('churn', axis=1), df_dic.churn.ravel())

print('After OverSampling, the shape of train_X: {}'.format(df_dic_res.shape))
print('After OverSampling, the shape of train_y: {}'.format(y_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_res==0)))

In [ ]:
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split (df_dic_res, 
                                                     y_res,
                                                     test_size = 0.2,
                                                     random_state = 0,
                                                     stratify = y_res)

In [ ]:
classifier5 = RandomForestClassifier(criterion = 'entropy', n_estimators= 500, max_depth = 16, max_features= 'auto').fit(X_train_res, y_train_res)
pred5 = classifier5.predict(X_test_res)

In [ ]:
print(classification_report(y_test_res,pred5))

In [ ]:
saca_metricas(y_test_res,pred5)

In [ ]:
classifier5.feature_importances_

In [ ]:
sorted_idx = classifier5.feature_importances_.argsort()
plt.barh(X_train_res.columns, classifier5.feature_importances_[sorted_idx])
plt.xlabel("Random Forest Feature Importance")

In [ ]:
feat_importances = pd.Series(classifier5.feature_importances_, index=X_train_res.columns)
feat_importances.sort_values().plot(kind='barh')

In [ ]:
pkl_filename = "modelo1.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(classifier5, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

In [ ]:
!jt -t monokai